# Jacobi and Gauss-Seidel

Numba accelerated.

In [1]:
import numpy as np
from numpy.linalg import norm
from numba import njit, prange

In [2]:
def correct(f, A, b):
    x = f(A, b)
    return np.allclose(np.dot(A, x), b)

## Jacobi algorithm

The first implementation is a rather reasonable numpy implementation of the Jacobi algorithm. The second is precisely the same code, but JIT compiled by numba. The third is loop based and numba accelerated. The fourth is loop based and numba accelerated with the parallel flag on (numba will attempt to parallelize loops marked with `prange`).

In [3]:
MAXIT = 1000


def jacobi_np(A, b):
    D = np.diag(A)
    T = (np.diag(D) - A) / np.expand_dims(D, 1)
    c = b / D

    x = np.zeros_like(b)
    for _ in range(MAXIT):

        ox = x.copy()
        x[:] = np.dot(T, x) + c
        
        if norm(x - ox) / norm(x) < 1e-9:
            break
    
    return x


def jacobi_loop(A, b):
    x = b.copy()

    for _ in range(MAXIT):

        z = np.empty(len(b))
        for i in prange(b.shape[0]):
            z[i] = (- np.sum(A[i] * x) + A[i, i] * x[i] + b[i]) / A[i, i]
        
        ox = x.copy()
        x[:] = z

        if norm(x - ox) / norm(x) < 1e-9:
            break

    return x


jacobi_np_numba = njit(fastmath=True, nogil=True)(jacobi_np)
jacobi_loop_numba = njit(fastmath=True, nogil=True)(jacobi_loop)
jacobi_loop_numba_parallel = njit(parallel=True, fastmath=True, nogil=True)(jacobi_loop)

In [7]:
import numpy.random as rng

n = 5000
A = rng.normal(size=(n, n)) + np.eye(n) * 100
b = rng.normal(size=(n))

assert correct(jacobi_np, A, b)
assert correct(jacobi_np_numba, A, b)
assert correct(jacobi_loop_numba, A, b)
assert correct(jacobi_loop_numba_parallel, A, b)

In [5]:
%timeit jacobi_np(A, b)

4.18 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%prun jacobi_np(A, b)

         1857 function calls (1734 primitive calls) in 0.341 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  307/184    0.213    0.001    0.228    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.110    0.110    0.338    0.338 <ipython-input-3-909bf341ce58>:4(jacobi_np)
        2    0.013    0.007    0.013    0.007 twodim_base.py:229(diag)
        1    0.002    0.002    0.341    0.341 <string>:1(<module>)
      120    0.001    0.000    0.002    0.000 linalg.py:2362(norm)
      120    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
      180    0.000    0.000    0.213    0.001 <__array_function__ internals>:2(dot)
      120    0.000    0.000    0.003    0.000 <__array_function__ internals>:2(norm)
       60    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' 

In [142]:
%timeit jacobi_np_numba(A, b)

345 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [143]:
%timeit jacobi_loop_numba(A, b)

1.26 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
%timeit jacobi_loop_numba_parallel(A, b)

274 ms ± 7.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Gauss-Seidel algorithm

The first implementation is a rather reasonable numpy implementation of the Jacobi algorithm. The second is precisely the same code, but JIT compiled by numba. The third is loop based and numba accelerated. The fourth is loop based and numba accelerated with the parallel flag on (numba will attempt to parallelize loops marked with `prange`).

The attentive reader will observe the fourth implementation is not a correct implementation of Gauss-Seidel. Indeed, it is impossible to parallelize Gauss-Seidel and preserve the same semantics. But it seems that the (potential) disadvantages to embracing disorder in the updates of `x` is outweighed by the benefits of parallelization.

In [145]:
MAXIT = 1000


def gauss_seidel_np(A, b):
    D = np.diag(A)
    T = (np.diag(D) - A) / np.expand_dims(D, 1)
    c = b / D

    x = b.copy()
    for _ in range(MAXIT):

        ox = x.copy()
        for i in range(len(b)):
            x[i] = np.sum(T[i] * x) + c[i]
        
        if norm(x - ox) / norm(x) < 1e-9:
            break
    
    return x


@njit
def gauss_seidel_np_numba(A, b):
    D = np.diag(A)
    T = (np.diag(D) - A) / np.expand_dims(D, 1)
    c = b / D

    x = b.copy()
    for _ in range(MAXIT):

        ox = x.copy()
        for i in range(len(b)):
            x[i] = np.sum(T[i] * x) + c[i]
        
        if norm(x - ox) / norm(x) < 1e-9:
            break
    
    return x


@njit(fastmath=True, nogil=True)
def gauss_seidel_loop_numba(A, b):
    x = b.copy()

    for _ in range(MAXIT):

        ox = x.copy()
        for i in prange(b.shape[0]):
            x[i] = (- np.sum(A[i] * x) + A[i, i] * x[i] + b[i]) / A[i, i]

        if norm(x - ox) / norm(x) < 1e-9:
            break

    return x


@njit(parallel=True, nogil=True)
def gauss_seidel_loop_numba_parallel(A, b):
    x = b.copy()

    for _ in range(MAXIT):

        ox = x.copy()
        for i in prange(b.shape[0]):
            x[i] = (- np.sum(A[i] * x) + A[i, i] * x[i] + b[i]) / A[i, i]

        if norm(x - ox) / norm(x) < 1e-9:
            break

    return x

In [146]:
assert correct(gauss_seidel_np, A, b)
assert correct(gauss_seidel_np_numba, A, b)
assert correct(gauss_seidel_loop_numba, A, b)
assert correct(gauss_seidel_loop_numba_parallel, A, b)

None

In [147]:
%timeit gauss_seidel_np(A, b)

1.03 s ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [148]:
%timeit gauss_seidel_np_numba(A, b)

761 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
%timeit gauss_seidel_loop_numba(A, b)

709 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [150]:
%timeit gauss_seidel_loop_numba_parallel(A, b)

215 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Results on MBP M1

| Implementation | Running time |
| -------------- | ------------ |
| Jacobi numpy   | 300 ms       |
| Jacobi numpy njit | 345 ms    |
| Jacobi loop njit | 1.26 s     |
| Jacobi loop njit parallel | 274 ms |
| GS numpy       | 1.03 s       |
| GS numpy njit  | 761 ms       |
| GS loop njit   | 709 ms       |
| GS loop njit parallel | 215 ms |